<h2 align="center">Movie Review Prediction: A Sentiment Analysis Case Study</h2>

### Introduction
___

- IMDB Movie Reviews Dataset
- https://ai.stanford.edu/~amaas/data/sentiment
- Contains 25000 positive and 25000 negative reviews
<img src="https://i.imgur.com/lQNnqgi.png" align="center">
- Contains at most reviews per movie
- At least 7 stars out of 10 $\rightarrow$ positive (label = 1)
- At most 4 stars out of 10 $\rightarrow$ negative (label = 0)
- 50/50 train/test split
- Evaluation accuracy

<b>Features: Bag of 1-grams with TF-IDF values</b>:
- Extremely sparse feature matrix - close to 97% are zeros

<b>Model: Logistic Regression</b>:
- $p(y = 1|x) = \sigma(w^{T}x)$
- Linear classification model
- Weights can be interpreted
- Can handle sparse data
- Fast to train
<img src="https://i.imgur.com/VieM41f.png" align="center" width=500 height=500>

### Loading the Dataset

In [1]:
import pandas as pd
df = pd.read_csv("movie_data.csv")
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [2]:
df['review'][0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

### Transforming Documents into Feature Vectors

Below, we will call the fit_transform method on CountVectorizer. This will construct the vocabulary of the bag-of-words model and transform the following three sentences into sparse feature vectors:
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two


In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
'The weather is sweet',
'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [4]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [5]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Raw term frequencies: *tf (t,d)* - The number of times a term t occurs in a document *d*.

### Word Relevancy using Term Frequency - Inverse Document Frequency

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

where $n_d$ is the total number of documents, and df(d, t) is the number of documents d that contain the term t.

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision = 2)
tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf = True)
print(tfidf.fit_transform(bag).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [7]:
print(tfidf.fit_transform(bag))
np.set_printoptions(precision = 2)

  (0, 6)	0.4337078595086741
  (0, 4)	0.5584778353707552
  (0, 3)	0.5584778353707552
  (0, 1)	0.4337078595086741
  (1, 8)	0.5584778353707552
  (1, 6)	0.4337078595086741
  (1, 5)	0.5584778353707552
  (1, 1)	0.4337078595086741
  (2, 8)	0.191038921512224
  (2, 7)	0.25119322405394995
  (2, 6)	0.296717529270996
  (2, 5)	0.191038921512224
  (2, 4)	0.191038921512224
  (2, 3)	0.191038921512224
  (2, 2)	0.5023864481078999
  (2, 1)	0.44507629390649395
  (2, 0)	0.5023864481078999


The equations for the idf and tf-idf that are implemented in scikit-learn are:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$
The tf-idf equation that is implemented in scikit-learn is as follows:

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

### Data Preparation

In [8]:
df.loc[0, 'review'][-100:] 

'lack of the emotion in the dramatization. My vote is seven.<br /><br />Title (Brazil): Not Available'

In [9]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [10]:
preprocessor(df.loc[0, 'review'][-100:])

'lack of the emotion in the dramatization my vote is seven title brazil not available'

In [11]:
preprocessor("</a>This :) is a :( test :-)!")

'this is a test :) :( :)'

In [12]:
df['review'] = df['review'].apply(preprocessor)

### Tokenization of Documents

In [13]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [14]:
def tokenizer(text):
    return text.split()

In [15]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [16]:
tokenizer("Runners like running and thus they run")

['Runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [17]:
tokenizer_porter("Runners like running and thus they run")

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Aniket
[nltk_data]     Das\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter("a runner likes running and runs a lot") if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

### Transform Text Data into TF-IDF Vectors

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, preprocessor = None,
                        tokenizer = tokenizer_porter, use_idf = True, norm = 'l2', smooth_idf = True)
y = df.sentiment.values
x = tfidf.fit_transform(df.review)

In [21]:
print(x)
print(y)

  (0, 5658)	0.06189823027375796
  (0, 9577)	0.07823676210624922
  (0, 65568)	0.046464630963170384
  (0, 57755)	0.06732110251347408
  (0, 70298)	0.06401311587993956
  (0, 44310)	0.02604551168313056
  (0, 19458)	0.056551569927559366
  (0, 21062)	0.04742484344780662
  (0, 36681)	0.045976609223807606
  (0, 64857)	0.02253272212236225
  (0, 48229)	0.07446379790252702
  (0, 16842)	0.03752344664304985
  (0, 37122)	0.03997250716439503
  (0, 58586)	0.030954015728543647
  (0, 57041)	0.05767283790521518
  (0, 15668)	0.05614627309394783
  (0, 30048)	0.08087282076115677
  (0, 31016)	0.02911304675396796
  (0, 18464)	0.09558037156130314
  (0, 1861)	0.049569736385307206
  (0, 49074)	0.13116665841692368
  (0, 14865)	0.07136715463323197
  (0, 59936)	0.1249170429237324
  (0, 31029)	0.038642227983825925
  (0, 64639)	0.027822474705817047
  :	:
  (49999, 61759)	0.13972583224617544
  (49999, 7059)	0.12172309100607187
  (49999, 52346)	0.3150038171059168
  (49999, 18173)	0.17006882370501028
  (49999, 64995)	0.1

### Document Classification using Logistic Regression

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state = 1, test_size = 0.5, shuffle = False)

In [23]:
import pickle
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv = 5, scoring = 'accuracy', random_state = 0, n_jobs = -1, verbose = 3, max_iter = 300).fit(X_train, Y_train)
save_model = open('save_model.sav', 'wb')
pickle.dump(clf, save_model)
save_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   48.0s remaining:  1.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   50.1s finished


### Model Evaluation

In [24]:
filename = 'save_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [25]:
saved_clf.score(X_test, Y_test)

C:\Anaconda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.89608